# 💰 Gold Sales Dashboard — Overview & Explanation

This script creates an **Excel dashboard** for gold sales management in 2025 with monthly sheets, a full-year summary, and charts — all dynamically linked and styled.

---

## ⚙️ What the Script Does

- 📅 **Generates monthly sheets** with all working days (Mon-Fri).
- ✍️ **Leaves daily sales data columns blank** for manual input:
  - 🏋️‍♂️ Grams Sold
  - 💵 Sale Amount
  - 📊 Price per Gram (auto-calculated)
  - 🛠️ Cost per Gram
  - 📈 Gross Profit (auto-calculated)
- 🧮 **Totals row per month** calculates sums and averages with formulas.
- 📊 **Monthly charts**:
  - 📉 Bar chart showing *Grams Sold* by date on X-axis.
  - 📈 Line chart showing *Gross Profit* by date on X-axis.
- 📋 **Full Year Data sheet** aggregates all days for review/filtering.
- 📑 **Summary sheet** compiles monthly totals with currency formatting and combined bar + line chart.
- 💾 **Saves** workbook to `~/Documents/Gold_Sales_Dashboard.xlsx`.

In [40]:
import pandas as pd
from datetime import datetime, timedelta
import os
import calendar
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Alignment, Font, Border, Side
from openpyxl.chart import BarChart, LineChart, Reference
from openpyxl.worksheet.table import Table, TableStyleInfo

def generate_month_data(year):
    all_month_data = []
    for month in range(1, 13):
        first_day = datetime(year, month, 1)
        last_day = datetime(year if month < 12 else year + 1, month % 12 + 1, 1) - timedelta(days=1)
        weekdays = [d for d in pd.date_range(first_day, last_day) if d.weekday() <= 5]
        month_data = [{
            "Date": d.strftime('%d/%m/%Y'), "Day": d.strftime('%A'),
            "Grams Sold": "", "Sale Amount": "",
            "Price per Gram": "", "Cost per Gram": "", "Gross Profit": ""
        } for d in weekdays]
        all_month_data.append((calendar.month_name[month], month_data))
    return all_month_data

year = 2025
data_2025 = generate_month_data(year)
month_dfs = {month: pd.DataFrame(data) for month, data in data_2025}
save_path = os.path.expanduser("~/Documents/Gold_Sales_Dashboard.xlsx")

month_workdays = {calendar.month_name[m]: len(df) for m, df in enumerate(month_dfs.values(), start=1)}

wb = Workbook()
wb.remove(wb.active)
full_year_data = []

for month, df in month_dfs.items():
    ws = wb.create_sheet(title=month)
    ws.append([f'{month[:3]}-25'])
    ws.merge_cells('A1:G1')
    a1 = ws['A1']
    a1.fill = PatternFill(start_color='FCE4E4', fill_type='solid')
    a1.font = Font(bold=True, size=14)
    a1.alignment = Alignment(horizontal="center")

    headers = ["Date", "Day", "Grams Sold", "Sale Amount", "Price per Gram", "Cost per Gram", "Gross Profit"]
    ws.append(headers)
    for c in ws[2]:
        c.fill = PatternFill(start_color='B6D7A8', fill_type='solid')
        c.font = Font(bold=True, size=14)
        c.alignment = Alignment(horizontal="center")

    for idx, row in df.iterrows():
        i = idx + 3
        ws.append([
            row['Date'], row['Day'], "", "",
            f'=IF(AND(C{i}="", D{i}=""), "", D{i}/C{i})',
            "",
            f'=IF(OR(C{i}="", D{i}="", F{i}=""), "", D{i}-(F{i}*C{i}))'
        ])
        full_year_data.append([row['Date'], row['Day']])

    total_row = 2 + len(df) + 1
    ws.append([
        "Totals", f"{len(df)} workdays",
        f'=SUM(C3:C{total_row - 1})',
        f'=SUM(D3:D{total_row - 1})',
        f'=AVERAGEIF(E3:E{total_row - 1}, "<>", E3:E{total_row - 1})',
        f'=AVERAGEIF(F3:F{total_row - 1}, "<>", F3:F{total_row - 1})',
        f'=SUM(G3:G{total_row - 1})'
    ])

    for col in range(1, 8):
        cell = ws.cell(row=total_row, column=col)
        cell.font = Font(bold=True, size=14)
        cell.fill = PatternFill(start_color='F4CCCC', fill_type='solid')
        cell.alignment = Alignment(horizontal="center")

    for row in ws.iter_rows(min_row=3, max_row=total_row, min_col=1, max_col=7):
        for cell in row:
            cell.fill = PatternFill(start_color='D9EAD3', fill_type='solid')
            cell.border = Border(left=Side(style='thin'), right=Side(style='thin'),
                                  top=Side(style='thin'), bottom=Side(style='thin'))
            cell.alignment = Alignment(horizontal="center")

    for col in ['A','B','C','D','E','F','G']:
        ws.column_dimensions[col].width = 16

    # Plain number formatting (no currency) for monthly sheets
    for row in ws.iter_rows(min_row=3, max_row=total_row-1, min_col=4, max_col=7):
        for cell in row:
            cell.number_format = '#,##0.00'

    # Grams Sold Bar Chart
    bar = BarChart()
    bar.title = f"{month} - Grams Sold"
    bar.y_axis.title = "Grams Sold"
    grams_data = Reference(ws, min_col=3, min_row=3, max_row=total_row-1)
    dates = Reference(ws, min_col=1, min_row=3, max_row=total_row-1)
    bar.add_data(grams_data, titles_from_data=False)
    bar.set_categories(dates)
    bar.legend = None
    ws.add_chart(bar, "I5")

    # Gross Profit Line Chart
    line = LineChart()
    line.title = f"{month} - Gross Profit"
    line.y_axis.title = "Gross Profit"
    profit_data = Reference(ws, min_col=7, min_row=3, max_row=total_row-1)
    line.add_data(profit_data, titles_from_data=False)
    line.set_categories(dates)
    line.legend = None
    ws.add_chart(line, "I20")

# Full Year Sheet
ws_all = wb.create_sheet("Full Year Data")
headers = ["Date", "Day", "Grams Sold", "Sale Amount", "Price per Gram", "Cost per Gram", "Gross Profit"]
ws_all.append(headers)
for month, df in month_dfs.items():
    for i, row in df.iterrows():
        ws_all.append([row['Date'], row['Day'], "", "", "", "", ""])

tbl = Table(displayName="FullYearTable", ref=f"A1:G{len(full_year_data)+1}")
tbl.tableStyleInfo = TableStyleInfo(name="TableStyleMedium9", showRowStripes=True)
ws_all.add_table(tbl)

for col in ['A','B','C','D','E','F','G']:
    ws_all.column_dimensions[col].width = 16

# Summary Sheet
ws_summary = wb.create_sheet("Summary")
summary_headers = ["Month", "Grams Sold", "Sale Amount", "Avg Price per Gram", "Avg Cost per Gram", "Gross Profit"]
ws_summary.append(summary_headers)

for cell in ws_summary[1]:
    cell.fill = PatternFill(start_color='B6D7A8', fill_type='solid')
    cell.font = Font(bold=True, size=14)
    cell.alignment = Alignment(horizontal="center")

for row_num, (month, workdays) in enumerate(month_workdays.items(), start=2):
    total_row = 2 + workdays + 1
    row = [
        month,
        f'={month}!C{total_row}',
        f'={month}!D{total_row}',
        f'={month}!E{total_row}',
        f'={month}!F{total_row}',
        f'={month}!G{total_row}'
    ]
    ws_summary.append(row)

# Format summary sheet numbers with currency
for row in ws_summary.iter_rows(min_row=2, max_row=13, min_col=2, max_col=6):
    for cell in row:
        cell.number_format = '₪#,##0.00'
        cell.font = Font(size=12)
        cell.alignment = Alignment(horizontal="center")
        cell.fill = PatternFill(start_color='D9EAD3', fill_type='solid')
        cell.border = Border(left=Side(style='thin'), right=Side(style='thin'),
                             top=Side(style='thin'), bottom=Side(style='thin'))

for col in ['A','B','C','D','E','F']:
    ws_summary.column_dimensions[col].width = 18

# Summary Chart
bar = BarChart()
line = LineChart()
bar.title = "Monthly Summary"
bar_data = Reference(ws_summary, min_col=2, max_col=2, min_row=1, max_row=13)
line_data = Reference(ws_summary, min_col=6, max_col=6, min_row=1, max_row=13)
cats = Reference(ws_summary, min_col=1, min_row=2, max_row=13)

bar.add_data(bar_data, titles_from_data=True)
line.add_data(line_data, titles_from_data=True)
bar.set_categories(cats)
line.set_categories(cats)
bar += line
line.y_axis.axId = 200
bar.y_axis.title = "Grams Sold"
line.y_axis.title = "Gross Profit"
ws_summary.add_chart(bar, "H3")

# Save
wb.save(save_path)
print(f"Excel file 'Gold_Sales_Dashboard.xlsx' has been generated successfully and saved to: {save_path}")


Excel file 'Gold_Sales_Dashboard.xlsx' has been generated successfully and saved to: /Users/mohab/Documents/Gold_Sales_Dashboard.xlsx


# 📝 Sale Log

This sheet is an open log where **all gold sales** are recorded with **automatic date and time stamping**. ⏰📅

It allows **multiple entries throughout the day** and feeds data directly into the **Gold Sales Dashboard** for easy summary and analysis 📊.

---

We use the following script to integrate the Sale Log into the document created earlier.  

**Purpose:**  
- To record every sale made during the day  
- Automatically detect and fill in the current date and time  
- Sync all daily data with monthly sheets at the press of a button 🔄  

This makes tracking sales dynamic, effortless, and error-free! 🚀

In [41]:
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment
import os

# Path to your existing file
file_path = os.path.expanduser("~/Documents/Gold_Sales_Dashboard.xlsx")

# Load existing workbook
wb = load_workbook(file_path)

# Create "Sale Log" sheet as the first sheet
if "Sale Log" not in wb.sheetnames:
    ws = wb.create_sheet(title="Sale Log", index=0)
else:
    ws = wb["Sale Log"]

# Headers
headers = ["Date", "Time", "Grams Sold", "Sale Amount", "Price per Gram", "Cost per Gram", "Gross Profit"]
ws.append(headers)

# Adjust column widths for readability
for col_letter in ['A','B','C','D','E','F','G']:
    ws.column_dimensions[col_letter].width = 16

# Set formulas for rows 2 to 31 (30 rows)
for row in range(2, 32):
    ws[f"E{row}"] = f'=IF(AND(C{row}<>"",D{row}<>""), D{row}/C{row}, "")'
    ws[f"G{row}"] = f'=IF(AND(C{row}<>"",D{row}<>"",F{row}<>""), D{row}-(F{row}*C{row}), "")'

# Style header with font size 14, bold, and center alignment
for cell in ws[1]:
    cell.font = Font(bold=True, size=14)
    cell.alignment = Alignment(horizontal="center")

# Save workbook
wb.save(file_path)
print(f"'Sale Log' sheet added/updated as first sheet in: {file_path}")


'Sale Log' sheet added/updated as first sheet in: /Users/mohab/Documents/Gold_Sales_Dashboard.xlsx


## 📈 Daily Sales Tracker Automation Setup (Google Sheets + Apps Script)

After creating your **monthly sheets** (named `January`, `February`, etc.) and the main `Sale Log` sheet, follow this process to automate your sales tracking and data aggregation.

---

### ✅ How It Works

1. **Sheet Setup**
   - Create a `Sale Log` sheet where you input your daily sales (with columns like Date, Time, Grams, Amount, etc.).
   - Create one sheet per month (e.g., `June`, `July`) where column A is pre-filled with each day of the month in `dd/MM/yyyy` format.

2. **Script Integration**
   - In your Google Sheet, go to:
     ```
     Extensions → Apps Script
     ```
   - Remove any existing code and **paste the following script**:

> **Note:** The current script uses the timezone setting:  
> ```js
> const timeZone = "Asia/Jerusalem";
> ```  
> Make sure to **update this timezone** if you recreate the system for a different region, so date and time stamping matches your local time.

   ```javascript
   function onEdit(e) {
     if (!e) return;

     const sheet = e.range.getSheet();

     if (sheet.getName() !== "Sale Log") return;

     const editedColumn = e.range.getColumn();
     const editedRow = e.range.getRow();

     if (editedColumn === 3) { // Grams Sold column
       const value = e.range.getValue();
       if (value === "") return;

       const now = new Date();
       const timeZone = "Asia/Jerusalem";

       const dateCell = sheet.getRange(editedRow, 1);
       if (dateCell.getValue() === "") {
         dateCell.setValue(Utilities.formatDate(now, timeZone, "yyyy-MM-dd"));
       }

       const timeCell = sheet.getRange(editedRow, 2);
       if (timeCell.getValue() === "") {
         timeCell.setValue(Utilities.formatDate(now, timeZone, "HH:mm:ss"));
       }
     }
   }

   function onOpen() {
     SpreadsheetApp.getUi()
       .createMenu("Daily Sync")
       .addItem("Send to Monthly Sheet", "sendToMonthlySheet")
       .addToUi();
   }

   function sendToMonthlySheet() {
     const ss = SpreadsheetApp.getActiveSpreadsheet();
     const logSheet = ss.getSheetByName("Sale Log");
     if (!logSheet) {
       SpreadsheetApp.getUi().alert("Sale Log sheet not found!");
       return;
     }

     const data = logSheet.getDataRange().getValues();
     const timeZone = "Asia/Jerusalem";
     const today = new Date();

     const dateStringForLog = Utilities.formatDate(today, timeZone, "yyyy-MM-dd");
     const monthSheetName = Utilities.formatDate(today, timeZone, "MMMM");
     const monthSheet = ss.getSheetByName(monthSheetName);
     if (!monthSheet) {
       SpreadsheetApp.getUi().alert(`Sheet "${monthSheetName}" does not exist.`);
       return;
     }

     const rows = data.slice(1).filter(row => {
       return row[0] && Utilities.formatDate(new Date(row[0]), timeZone, "yyyy-MM-dd") === dateStringForLog;
     });

     if (rows.length === 0) {
       SpreadsheetApp.getUi().alert("No data found for today in Sale Log.");
       return;
     }

     let totalGrams = 0, totalAmount = 0, totalCost = 0;

     rows.forEach(row => {
       const grams = parseFloat(row[2]) || 0;
       const amount = parseFloat(row[3]) || 0;
       const cost = parseFloat(row[5]) || 0;

       totalGrams += grams;
       totalAmount += amount;
       totalCost += cost * grams;
     });

     const dateStringForMonthSheet = Utilities.formatDate(today, timeZone, "dd/MM/yyyy");
     const lastRow = monthSheet.getLastRow();
     let targetRow = -1;

     for (let r = 2; r <= lastRow; r++) {
       const cellValue = monthSheet.getRange(r, 1).getDisplayValue().trim();
       if (cellValue === dateStringForMonthSheet) {
         targetRow = r;
         break;
       }
     }

     if (targetRow === -1) {
       SpreadsheetApp.getUi().alert(`Date ${dateStringForMonthSheet} not found in "${monthSheetName}" sheet column A.`);
       return;
     }

     const avgCostPerGram = totalGrams > 0 ? totalCost / totalGrams : 0;
     const avgPricePerGram = totalGrams > 0 ? totalAmount / totalGrams : 0;
     const grossProfit = totalAmount - (avgCostPerGram * totalGrams);

     // Round all to nearest integer (no decimals)
     const totalGramsRounded = Math.round(totalGrams);
     const totalAmountRounded = Math.round(totalAmount);
     const avgPricePerGramRounded = Math.round(avgPricePerGram);
     const avgCostPerGramRounded = Math.round(avgCostPerGram);
     const grossProfitRounded = Math.round(grossProfit);

     monthSheet.getRange(targetRow, 3).setValue(totalGramsRounded);
     monthSheet.getRange(targetRow, 4).setValue(totalAmountRounded);
     monthSheet.getRange(targetRow, 5).setValue(avgPricePerGramRounded);
     monthSheet.getRange(targetRow, 6).setValue(avgCostPerGramRounded);
     monthSheet.getRange(targetRow, 7).setValue(grossProfitRounded);

     // Clear Sale Log but keep formulas as before
     const lastRowLog = logSheet.getLastRow();
     const lastColLog = logSheet.getLastColumn();

     if (lastRowLog > 1) {
       const dataRange = logSheet.getRange(2, 1, lastRowLog - 1, lastColLog);
       const values = dataRange.getValues();

       // Clear all values
       for (let i = 0; i < values.length; i++) {
         for (let j = 0; j < values[i].length; j++) {
           values[i][j] = "";
         }
       }
       dataRange.setValues(values);

       // Restore formulas in columns E (5) and G (7)
       for (let i = 0; i < lastRowLog - 1; i++) {
         const rowIndex = i + 2;
         logSheet.getRange(rowIndex, 5).setFormula(`=IF(AND(C${rowIndex}<>"",D${rowIndex}<>""), D${rowIndex}/C${rowIndex}, "")`);
         logSheet.getRange(rowIndex, 7).setFormula(`=IF(AND(C${rowIndex}<>"",D${rowIndex}<>"",F${rowIndex}<>""), D${rowIndex}-(F${rowIndex}*C${rowIndex}), "")`);
       }
     }

     SpreadsheetApp.getUi().alert(`Daily data sent to '${monthSheetName}'. Log cleared and formulas restored.`);
   }


💾 **Save & Authorize**  
Click the floppy disk icon or press Ctrl + S to save.  

Reload the sheet and authorize the script when prompted.  

📆 **Daily Use**  
As you enter the Grams Sold (Column C), the script automatically fills in:  
- the Date (Column A)  
- and the Time (Column B)  

At the end of the day, go to the menu and click:  

    Daily Sync → Send to Monthly Sheet


This will:  
- Transfer the day's data (total grams, revenue, cost, profit) into the correct date row of the matching monthly sheet.  
- Clear the Sale Log.  
- Restore all formulas for the next day's entries.  

🧠 **Why This Is Useful**  
- Ensures consistent and accurate daily tracking.  
- Avoids manual entry errors for dates and formulas.  
- Makes end-of-month analysis effortless with clean, summarized data.

### Final Remarks

This setup can be easily replicated each year by simply updating the starting year in your sheets (e.g., changing '2025' to the new year) and repeating this process, which takes very little time to recreate.

Notably, if you prefer, you can skip the Sale Log entirely and record only the final daily data directly into the monthly sheets by ignoring the daily entry steps (Step 2 and 3). This simplifies data entry but removes the detailed log of individual sales.

Use whichever approach best fits your workflow!
